<h1>Toxic Comments Classification Challange</h1><br>
Implementasi menggunakan <strong>Word2Vec</strong> dan <strong>Convolutional Neural Networks</strong><br>
https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

In [115]:
import os
import sys
import re
import json
import mysql.connector
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from tflearn.data_utils import pad_sequences

Berikut dilakukan pendefinisian path untuk file-file yang digunakan

- Corpus yang digunakan adalah 100milyar kata yang diambil dari Google News. Pada corpus ini dijalankan algoritma Word2Vec yang menghasilkan word embeddings. Setiap kata terdiri dari vektor berdimensi 300. File bisa di-download di https://code.google.com/archive/p/word2vec/

In [2]:
data_path = 'data/'
corpus = 'GoogleNews-vectors-negative300.bin'

TRAIN_DATA_FILE = data_path + 'train.csv'
TEST_DATA_FILE = data_path + 'test.csv'

In [3]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

Melakukan Preprocessing pasa setiap teks (DONE - comments_np.npy)
1. Melakukan tokenisasi, memisahkan setiap kata
2. Membuang kata-kata yang termasuk dalam stopwords
3. Membuang string yang mengandung karakter selain huruf dan tanda petik satu

In [4]:
def preprocess(string):
    raw = string.split(' ')
    remover = re.compile("[^a-zA-Z']")
    
    token = []
    
    for i in raw:
        if i in stopwords.words('english'):
            continue
        term = remover.sub('', i).lower()
        token.append(term)
    
    row = filter(None, token)
    
    return np.array(row)

comments_text = train['comment_text'].values
comments = []
for comm in comments_text:
    tmp = comm.replace('\n', ' ')
    tmp = comm.replace('.', '. ')
    tmp = preprocess(comm)
    
    comments.append(tmp)
    
comments_np = np.array(comments)
np.save('comments_np.npy', comments_np)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


Melakukan proses untuk mendapatkan <strong>HOT-VECTOR</strong> pada setiap kelas sentiment<br>
Kelas-kelasnya adalah:
1. toxic
2. severe_toxic
3. obscene
4. threat
5. insult
6. identity_hate

In [37]:
toxic_cls = train["toxic"].values
severe_toxic_cls = train["severe_toxic"].values
obscene_cls = train["obscene"].values
threat_cls = train["threat"].values
insult_cls = train["insult"].values
identity_hate_cls = train["identity_hate"].values

comments_cls_list = []
for i in range(len(comments_np)):
    hot_vec_list = []
    
    hot_vec_list.append(toxic_cls[i])
    hot_vec_list.append(severe_toxic_cls[i])
    hot_vec_list.append(obscene_cls[i])
    hot_vec_list.append(threat_cls[i])
    hot_vec_list.append(insult_cls[i])
    hot_vec_list.append(identity_hate_cls[i])
    
    hot_vec_np = np.array(hot_vec_list)
    comments_cls_list.append(hot_vec_np)
    
comments_cls_np = np.array(comments_cls_list)
np.save("comments_cls_np.npy", comments_cls_np)

In [4]:
# LOAD FROM CHECKPOINTS
comments_np = np.load("comments_np.npy")
# comments_cls_np = ("comments_cls_np.npy")

In [11]:
print "Mencari comment dengan kata terpanjang..."
max_comment_len = -1
for c in comments_np:
    if len(c) > max_comment_len:
        max_comment_len = len(c)
print "Jumlah kata pada comment terpanjang = " + str(max_comment_len)

Mencari comment dengan kata terpanjang...
Jumlah kata pada comment terpanjang = 1380


In [12]:
embedding_size = 300
max_feature = embedding_size * max_comment_len

In [3]:
word_vectors = KeyedVectors.load_word2vec_format(corpus, binary=True)
word_vectors.save_word2vec_format('w2v.txt', binary=False)

db = mysql.connector.connect(user="root", database="wor2vec")
cursor = db.cursor()

pbar = tqdm(total=3000000)
with open("w2v.txt") as f:
    try:
        for line in f:
            wv = line.split('\n')
            wv = wv[0].split()
            key = wv[0]
            vec = wv[1:]
            json_vec = json.JSONEncoder().encode(vec)
            #decoded_vec = json.JSONDecoder().decode(json_vec)
            #vec_d = np.asarray(decoded_vec, dtype=np.float32)

            sql = "insert into word (word, vec) values('"+ key +"', '" + json_vec + "')"
            cursor.execute(sql)

            pbar.update(1)

        pbar.close()
        print "DB COMMIT..."
        db.commit()
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))
        print sql
        db.rollback()

print "SUCCESS"
db.close()
cursor.close()

In [111]:
db = mysql.connector.connect(user="root", database="wor2vec")
cursor = db.cursor(buffered=True)

In [116]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data

def padSequence(feature, max_feature):
    pad = np.zeros(max_feature-len(feature), dtype=np.float32)
    feature = np.concatenate([feature, pad])
    return feature

ftr_list = []
pbar = tqdm(total=len(comments_np), file=sys.stdout)
for cmt in comments_np:
    begin = True
    for word in cmt:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature = np.concatenate([feature, vec])
    ftr_list.append(padSequence(feature, max_feature))
    pbar.update(1)

pbar.close()
ftr_np = np.array(ftr_list)
np.save("ftr_np.npy", ftr_np)

  2%|▏         | 2114/95851 [03:44<3:05:30,  8.42it/s] 

KeyboardInterrupt: 